# Vendor Quote Analysis

This notebook analyzes supplier quote data to identify cost variations, lead time delays, and sourcing inefficiencies. It generates insights and outputs for Tableau dashboards.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Set up file paths
DATA_DIR = Path('../data')
VISUALS_DIR = Path('../visuals')
INPUT_FILE = DATA_DIR / 'Cleaned_Supplier_Quote_Data.xlsx'
VISUALS_DIR.mkdir(exist_ok=True)

# Load cleaned data
df = pd.read_excel(INPUT_FILE)
print('Data Shape:', df.shape)
df.head()

In [ ]:
# Analyze cost variation by part
cost_variation = df.groupby('Part_Number')['Unit_Cost'].agg(['mean', 'std', 'min', 'max']).reset_index()
cost_variation['Variation_Percent'] = ((cost_variation['max'] - cost_variation['min']) / cost_variation['mean'] * 100).round(2)
print('Cost Variation by Part:\n', cost_variation)

# Save to CSV for Tableau
cost_variation.to_csv(DATA_DIR / 'Cost_Variation_by_Part.csv', index=False)

In [ ]:
# Analyze lead time by vendor
lead_time_analysis = df.groupby('Vendor_Name').agg({
    'Lead_Time_Days': ['mean', 'count'],
    'Lead_Time_Flag': lambda x: (x == 'Delayed').sum()
}).reset_index()
lead_time_analysis.columns = ['Vendor_Name', 'Avg_Lead_Time', 'Quote_Count', 'Delayed_Quotes']
print('Lead Time Analysis:\n', lead_time_analysis)

# Save to CSV for Tableau
lead_time_analysis.to_csv(DATA_DIR / 'Lead_Time_Analysis.csv', index=False)

In [ ]:
# Visualize Unit Cost vs. MOQ
plt.figure(figsize=(10, 6))
for vendor in df['Vendor_Name'].unique():
    vendor_data = df[df['Vendor_Name'] == vendor]
    plt.scatter(vendor_data['MOQ'], vendor_data['Unit_Cost'], label=vendor, alpha=0.6)
plt.xlabel('MOQ')
plt.ylabel('Unit Cost (USD)')
plt.title('Unit Cost vs. MOQ by Vendor')
plt.legend()
plt.grid(True)
plt.savefig(VISUALS_DIR / 'Unit_Cost_vs_MOQ.png')
plt.show()

In [ ]:
# Generate summary insights
high_cost_parts = cost_variation[cost_variation['Variation_Percent'] > 15]
delayed_vendors = lead_time_analysis[lead_time_analysis['Delayed_Quotes'] > 0]
print('Parts with >15% Cost Variation:\n', high_cost_parts[['Part_Number', 'Variation_Percent']])
print('Vendors with Delayed Quotes:\n', delayed_vendors[['Vendor_Name', 'Delayed_Quotes', 'Avg_Lead_Time']])